%%html
<style> table {display: block} </style> 
# Digital Signal Processing

## Exercise 4

### Part 1: Non-Recursive Filters

#### Task 1: window functions
In contrast to a recursive filter, whose design is based on the transformation of continuous time IIR-Systems to discrete time IIR-systems, the design of non recursive (FIT) systems is based on the approximation of the desired frequency response of the discrete time system. Due to their constant group delay, mostly _linear phase_ filters are applied, whose design is treated in the following text.

FIR systems with linear phase are based on windowing with ideal low-pass-filters. To keep the linear phase constraint the window functions have to be symmetric.  
The most common window functions are the so-called cosine windows, which have the following form:

\begin{equation*}
f(k) = \left\{
    \begin{array}{l l}	
         a - b \cdot \textrm{cos}(\frac{2\pi k}{m})+c \cdot \textrm{cos}(\frac{4\pi k}{m}), & 0\leq k \leq m\\
         0,  & \text{else}
    \end{array}\right.\;.
\end{equation*}

| **window**  | **python function** | **a** | **b** | **c** |
|:------------|:--------------------|:-----:|:-----:|:-----:|
| rectangular | boxcar             | 1.0   | 0.0   | 0.0   |
| von Hann    | hann             | 0.5   | 0.5   | 0.0   |
| Hamming     | hamming             | 0.54  | 0.46  | 0.0   |
| Blackman    | blackman            | 0.42  | 0.5   | 0.08  |

Another window is the triangle- or Bartlett-window (`bartlett`).

**a)** Calculate window types above for $m=21$ and $m=42$. Plot the signals in time domain as well as their time discrete fourier transform. For this use the `ldtft()`-function. Compare the different window types according to their pass-band attenuation, attenuation of the first overshot and stop-band attenuation.

**Python Hints:**
```python
r = signal.boxcar(m)      # returns rectangular window
R, W = ldtft(r, 512)      # calculates dtft of window
```


**b)** The Kaiser-window represents a flexible window, which is defined as follows:

\begin{equation*}
f(k) = \left\{
    \begin{array}{l l}	
         \frac{I_0 \left[\beta \sqrt{1-(1-\frac{2}{m}k)^2}\right]}{I_0(\beta)}, & 0\leq k \leq m\\
         0,  & \text{else}
    \end{array}\right.\;.
\end{equation*}

Where $I_0(x)$ is a modified bessel function of the first kind, 0th order. The Scipy-function `kaiser(m,beta)` calculates this window. In contrast to the windows of task **1.a** the Kaiser window has two parameters: The length $m+1$ and a parameter $\beta$ to affect the slope.

By varying $m$ and $\beta$ the length and the shape of the window may be varied to each a compromise between the amplitude of the first overshot and the width of the main maximum. Calculate the Kaiser window for $m=21$ and $\beta = 0,3,6$. Calculate the respective spectra and plot them along with the time domain representation of the windows.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from functions.ldtft import ldtft 
# Your code goes here!


#### Task 2: design of non recursive filters using windowing techniques
The complete filter design by windowing an impulse response of a time shifted, ideal low pass filter is implemented in Scipy-function `firwin()`.

Calling `b=signal.firwin(numtaps,Wn)` generates `numtaps` filter coefficients of a low-pass with order `numtaps-1` and cut-off frequency `Wn`. The function call `b=signal.firwin(numtaps,Wn,pass_zero='highpass')` yields the corresponding high pass with a cut-off frequency `Wn`.

`b=signal.fir1(numtaps,[Wn1 Wn2])` yields a pass-band filter with cut-off frequencies `Wn1` and `Wn2`. The Hamming-Window will be used as a _default_. In addition the last parameter may be a vector with samples of an arbitrary window function: `b=signal.fir1(numtaps,Wn,window)`.

**a)** Design a low-pass filter by setting `numtaps=51` and `Wn=0.4`. Compare the absolute value of the spectrum for different window functions.

**Python Hints:**
View [1] for a list of all available window types.
```python
b = signal.firwin(51, 0.4, window='boxcar')  # Create rectangular window with firwin
W, H = signal.freqz(b, 1, 512)               # Calculate frequency response of b
```


**b)** Design a high-pass filter of order $33$ (`numtaps=34`) and cut-off frequency `Wn=0.4` using the Scipy-Function `firwin()` (Hamming-Window). Why does Python generate a warning?

**c)** Group delay and linearity of a system are closely connected. Calculate the group delay of the low-pass filter of task **2.a** (Hamming-Window). Use the Scipy-Function `signal.group_delay((b,1),nfft)`.

[1] https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
# Your code goes here!


#### Task 3: remez design (Chebyshev-approximation)

By designing IIR-Filters using Chebyshev-approximation, it was shown that this approximation delivers a great benefit because it utilizes the whole tolerance scheme. Such a design method (applying a Chebyshev-approximation) for FIR-Systems is implemented in Scipy's `remez()`-function (Parks-McClellan-algorithm). The approximation takes place in pass- and stop-band.  
The call of `b = signal.remez(numtabs,F,G)` designs a `numtabs-1`-th order filter, whose specification is given by vectors `F` and `G`.  
`F` includes a pair of frequencies (between 0 and 0.5), to set the pass or stop-bands and `G` is the desired band gain.

Example: `F = [0, 0.15, 0.25, 0.5] G = [1 0]`

<img src="functions/img/remez.png" width="500">

**a)** Calculate a filter with above frequency response. Experiment with different orders and count the number of extrema within the approximation interval.

**b)** The call `b = signal.remez(numtabs,F,G,W)` weights a desired frequency response where frequency bands with a lower weight are less accurately approximated than frequency bands with a higher weight. The number of weight within vector `W` is half the number of entries in `F` (for each frequency band one weighting factor).

The frequency band between 0.0 and 0.3 (pass-band) doesn't have to approximate the desired frequency response very closely. A higher weight should be used for the frequency band between 0.5 and 1. Calculate the filter again and compare the frequency response with the filter designed in task **3.a**.

**c)** Design the low-pass - high-pass combination as specified below, where the passbands should have a (by factor 4) higher weight than the stop-bands:

\begin{equation*}
\begin{array}{l l l l}	
         \textrm{passband 1:} & 0 - 0.2 & \textrm{stop band 1:} & 0.26 - 0.44\\
         \textrm{passband 2:} & 0.5 - 0.7 & \textrm{stop band 2:} & 0.76 - 1\\
         \textrm{filter order } m: & 128 & &
    \end{array}
\end{equation*}


In [1]:
import matplotlib.pyplot as plt
import numpy as np
# Your code goes here!


#### Task 4: design of a differentiator
The differentiation in time domain is equivalent to a multiplication with $j\Omega$ in frequency domain.  
The transfer function of an ideal differentiator is $H(e^{j\Omega}) = j \Omega$.  
Design a differentiator of order 21 and 22 by using Scipy's function `remez()` (see task **3**) and plot the absolute values of the transfer function as well as the impulse response of both designed differentiators. Set the filter type to differentiator by using `remez(numtaps,F,G,type='differentiator')`.
Which class of FIR-filters do these FIR-filters belong to?

In [2]:
import matplotlib.pyplot as plt
import numpy as np
# Your code goes here!